![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/25.Date_Normalizer.ipynb)

# 25. Date Normalizer

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
jsl.install()

In [ ]:
from johnsnowlabs import *

# Automatically load license data and start a session with all jars user has access to
spark = jsl.start(exclude_ocr = True)

## **Date Normalizer**

New Annotator that transforms chunks Dates to a normalized Date with format YYYY/MM/DD. This annotator identifies dates in chunk annotations and transforms those dates to the format YYYY/MM/DD. 



We will create texts containing dates in different formats:

In [2]:
dates = [
'She has a history of pericarditis and pericardectomy on 08/02/2018 and developed a cough with right-sided chest pain.' ,
'She has been receiving gemcitabine and she receives three cycles of this with the last one being given on 11/2018. ',
'She was last seen in the clinic on 11/01/2018by Dr. Y.',
'Chris Brown was discharged on 12Mar2021',
'Last INR was on Tuesday, Jan 30, 2018, and her INR was 2.3. 2. Amiodarone 100 mg p.o. daily. ',
'We reviewed the pathology obtained from the pericardectomy on 13.04.1999, which was diagnostic of mesothelioma', 
'A review of her CT scan on 3 April2020 prior to her pericardectomy, already shows bilateral plural effusions. ',
]

df_dates = spark.createDataFrame(dates,StringType()).toDF('text')

df_dates.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|She has a history of pericarditis and pericardectomy on 08/02/2018 and developed a cough with rig...|
|She has been receiving gemcitabine and she receives three cycles of this with the last one being ...|
|                                              She was last seen in the clinic on 11/01/2018by Dr. Y.|
|                                                             Chris Brown was discharged on 12Mar2021|
|       Last INR was on Tuesday, Jan 30, 2018, and her INR was 2.3. 2. Amiodarone 100 mg p.o. daily. |
|We reviewed the pathology obtained from the pericardectomy on 13.04.1999, which was diagnostic of...|
|A review of her CT scan on 3 April2020 prior to her pericardectomy, alre

In [3]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model trained on n2c2 (de-identification and Heart Disease Risk Factors Challenge) datasets)
clinical_ner = medical.NerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = medical.NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("date_chunk")\
    .setWhiteList(["DATE"])

date_normalizer = medical.DateNormalizer()\
    .setInputCols('date_chunk')\
    .setOutputCol('normalized_date')

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      date_normalizer])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_generic_augmented download started this may take some time.
[OK!]


In [4]:
result = model.transform(df_dates)

We are going to show how the date is normalized.

In [5]:
result_df = result.select("text",F.explode(F.arrays_zip(result.date_chunk.result, 
                                                        result.normalized_date.result)).alias("cols")) \
                  .select("text",F.expr("cols['0']").alias("date_chunk"),
                                 F.expr("cols['1']").alias("normalized_date"))
                  
result_df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------+---------------------+---------------+
|text                                                                                                                 |date_chunk           |normalized_date|
+---------------------------------------------------------------------------------------------------------------------+---------------------+---------------+
|She has a history of pericarditis and pericardectomy on 08/02/2018 and developed a cough with right-sided chest pain.|08/02/2018           |2018/08/02     |
|She has been receiving gemcitabine and she receives three cycles of this with the last one being given on 11/2018.   |11/2018              |2018/11/DD     |
|She was last seen in the clinic on 11/01/2018by Dr. Y.                                                               |11/01/2018by         |2018/11/01     |
|Chris Brown was discharged on 12Mar2021            

### Only date chunk


We are going to create a chunks dates with different formats:

In [6]:
dates = [
'08/02/2018',
'11/2018',
'11/01/2018',
'12Mar2021',
'Jan 30, 2018',
'13.04.1999', 
'3April 2020',
]



In [7]:
df_dates = spark.createDataFrame(dates,StringType()).toDF('ner_chunk')

We are going to transform that text to documents in spark-nlp.

In [8]:
document_assembler = nlp.DocumentAssembler().setInputCol('ner_chunk').setOutputCol('document')
documents_DF = document_assembler.transform(df_dates)

After that we are going to transform that documents to chunks.

In [9]:
chunks_df = nlp.map_annotations_col(documents_DF.select("document","ner_chunk"),
                    lambda x: [Annotation('chunk', a.begin, a.end, a.result, a.metadata, a.embeddings) for a in x], "document",
                    "chunk_date", "chunk")

In [10]:
chunks_df.select('chunk_date').show(truncate=False)

+---------------------------------------------------+
|chunk_date                                         |
+---------------------------------------------------+
|[{chunk, 0, 9, 08/02/2018, {sentence -> 0}, []}]   |
|[{chunk, 0, 6, 11/2018, {sentence -> 0}, []}]      |
|[{chunk, 0, 9, 11/01/2018, {sentence -> 0}, []}]   |
|[{chunk, 0, 8, 12Mar2021, {sentence -> 0}, []}]    |
|[{chunk, 0, 11, Jan 30, 2018, {sentence -> 0}, []}]|
|[{chunk, 0, 9, 13.04.1999, {sentence -> 0}, []}]   |
|[{chunk, 0, 10, 3April 2020, {sentence -> 0}, []}] |
+---------------------------------------------------+



Now we are going to normalize those chunks using the DateNormalizer.

In [11]:
date_normalizer = medical.DateNormalizer().setInputCols('chunk_date').setOutputCol('date')

In [12]:
date_normalized_df = date_normalizer.transform(chunks_df)

We are going to show how the date is normalized.

In [13]:
dateNormalizedClean = date_normalized_df.selectExpr("ner_chunk","date.result as dateresult")

dateNormalizedClean.withColumn("dateresult", dateNormalizedClean["dateresult"].getItem(0)).show(truncate=False)

+------------+----------+
|ner_chunk   |dateresult|
+------------+----------+
|08/02/2018  |2018/08/02|
|11/2018     |2018/11/DD|
|11/01/2018  |2018/11/01|
|12Mar2021   |2021/03/12|
|Jan 30, 2018|2018/01/30|
|13.04.1999  |1999/04/13|
|3April 2020 |2020/04/03|
+------------+----------+



## Relative Date

We can configure the `anchorDateYear`,`anchorDateMonth` and `anchorDateDay` for the relatives dates.

In [14]:
rel_dates = [
'next monday',
'today',
'next week'
]

rel_dates_df = spark.createDataFrame(rel_dates,StringType()).toDF('ner_chunk')

In [15]:
rel_documents_DF = document_assembler.transform(rel_dates_df)

rel_chunks_df = nlp.map_annotations_col(rel_documents_DF.select("document","ner_chunk"),
                    lambda x: [Annotation('chunk', a.begin, a.end, a.result, a.metadata, a.embeddings) for a in x], "document",
                    "chunk_date", "chunk")

In [16]:
rel_chunks_df.select('chunk_date').show(truncate=False)

+--------------------------------------------------+
|chunk_date                                        |
+--------------------------------------------------+
|[{chunk, 0, 10, next monday, {sentence -> 0}, []}]|
|[{chunk, 0, 4, today, {sentence -> 0}, []}]       |
|[{chunk, 0, 8, next week, {sentence -> 0}, []}]   |
+--------------------------------------------------+



In the following example we will use as a relative date 2021/02/16, to make that possible we need to set up the `anchorDateYear` to 2021, the `anchorDateMonth` to 2 and the `anchorDateDay` to 16. We will show you the configuration with the following example.

In [17]:
rel_date_normalizer = medical.DateNormalizer()\
                        .setInputCols('chunk_date')\
                        .setOutputCol('date')\
                        .setAnchorDateDay(16)\
                        .setAnchorDateMonth(2)\
                        .setAnchorDateYear(2021)

In [18]:
rel_date_normalized_df = rel_date_normalizer.transform(rel_chunks_df)
relDateNormalizedClean = rel_date_normalized_df.selectExpr("ner_chunk","date.result as dateresult","date.metadata as metadata")
relDateNormalizedClean.withColumn("dateresult", relDateNormalizedClean["dateresult"].getItem(0))\
                      .withColumn("metadata", relDateNormalizedClean["metadata"].getItem(0)['normalized']).show(truncate=False)

+-----------+----------+--------+
|ner_chunk  |dateresult|metadata|
+-----------+----------+--------+
|next monday|2021/02/22|true    |
|today      |2021/02/16|true    |
|next week  |2021/02/23|true    |
+-----------+----------+--------+



As you see the relatives dates like `next monday` , `today` and `next week` takes the `2021/02/16` as reference date.
